In [ ]:
import hmftpy as hmf
from hmftpy.plaquettes.triangular import plaq12, plaq12z, plaq9p, plaq9d, plaq3
from hmftpy.operators import mf_ops, inner_hamiltonian, periodic_hamiltonian
from hmftpy import do_hmft
from hmftpy.plot_spins import draw_tri_lattice, draw_cluster, plot_spins
from quspin.basis import spin_basis_1d
from quspin.operators import quantum_operator
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

plaq = plaq12
L = plaq['L']
# basis = spin_basis_1d(L, pauli=0, Nup=(5, 6, 7))
basis = spin_basis_1d(L, pauli=0)


ops = hmf.operators.mf_ops(plaq, basis)
maxit = 50

In [ ]:
A, B, C = plaq['n_nearest_sublattice']
sl_mag_ops = {'A': {}, 'B': {}, 'C': {}}

dirs = ['x', 'y', 'z']
sls = {'A': A, 'B': B, 'C': C}
for d in dirs:
    for sl in sls:
        ind_lst = []
        op_lst = [[d, [[1, i] for i in sls[sl]]]]
        sl_mag_ops[sl][d] = quantum_operator({'static': op_lst}, basis=basis)
        
def get_mags(v, sl_ops, ind, mag_array, mag_2_array, mag_4_array):
    mags ={'A': {}, 'B': {}, 'C': {}}
    for d in dirs:
        for sl in sls:
            mag_array[sl][d][ind] = np.real(sl_ops[sl][d].expt_value(v))
            m2v = sl_ops[sl][d].dot(sl_ops[sl][d].dot(v))
            mag_2_array[sl][d][ind] = np.real(np.vdot(v, m2v))
            mag_4_array[sl][d][ind] = np.real(np.vdot(v, sl_ops[sl][d].dot(sl_ops[sl][d].dot(m2v))))

# Energy, sublattice magnetization, and Binder ratios

Let's calculate energy going left to right and right to left using HMFT.

In [ ]:
J2s = np.linspace(.3, 0., 50)

l = len(J2s)
labels = ['inner', 'periodic', 'lr', 'rl']
energies = {lab: np.zeros(l) for lab in labels}
mfs = {lab: [None for i in range(l)] for lab in labels}
cvgs = {lab: np.zeros(l) for lab in labels}

sl_mags = {lab: {s: {d: np.zeros(l) for d in dirs} for s in sls} for lab in labels}
sl_4_mags = {lab: {s: {d: np.zeros(l) for d in dirs} for s in sls} for lab in labels}
sl_2_mags = {lab: {s: {d: np.zeros(l) for d in dirs} for s in sls} for lab in labels}

In [ ]:
print('Running right to left')
mf0 = None
for i, J2 in enumerate(tqdm(J2s)):
    interactions = {'local': {},
                    'nearest': {'xx': 1, 'yy': 1, 'zz': 0},
                    'n_nearest': {'xx': J2, 'yy': J2, 'zz': 0},
                    'n_n_nearest': {}}
    Hi = inner_hamiltonian(plaq, interactions, basis, every_other=True)
    e, v = Hi.eigsh(k=1, which='SA', tol=10**-12)
    energies['inner'][i] = e[0]
    get_mags(v[:,0], sl_mag_ops, i, 
             sl_mags['inner'], sl_2_mags['inner'], sl_4_mags['inner'])

    
    Hp = periodic_hamiltonian(plaq, interactions, basis, every_other=True)
    e, v = Hp.eigsh(k=1, which='SA', tol=10**-12)
    energies['periodic'][i] = e[0]
    get_mags(v[:,0], sl_mag_ops, i, 
             sl_mags['periodic'], sl_2_mags['periodic'], sl_4_mags['periodic'])
    
    energies['rl'][i], v, mfs['rl'][i], cvgs['rl'][i] = do_hmft(plaq, interactions, basis, 
                                                                max_iter=maxit, mf0=mf0, 
                                                                ops=ops, Hi=Hi, hmft_tol=10**-8,
                                                                lanczos_tol=10**-11,
                                                                mf_cvg=False, every_other=True)
    mf0 = mfs['rl'][i]
    get_mags(v, sl_mag_ops, i, 
             sl_mags['rl'], sl_2_mags['rl'], sl_4_mags['rl'])
    
mf0 = None
print('Running left to right')

for i, J2 in enumerate(tqdm(J2s[::-1])):
    interactions = {'local': {},
                    'nearest': {'xx': 1, 'yy': 1, 'zz': 0},
                    'n_nearest': {'xx': J2, 'yy': J2, 'zz': 0},
                    'n_n_nearest': {}}
    try:
        energies['lr'][i], v, mfs['lr'][i], cvgs['lr'][i] = do_hmft(plaq, interactions, basis, 
                                                                    max_iter=maxit, mf0=mf0, 
                                                                    ops=ops, hmft_tol=10**-8,
                                                                    lanczos_tol=10**-11,
                                                                    mf_cvg=False, every_other=True)
    except:
        print('Failure at J2 = {}. Probably 0 mfs. Retrying with random MF'.format(J2))
        energies['lr'][i], v, mfs['lr'][i], cvgs['lr'][i] = do_hmft(plaq, interactions, basis, 
                                                                    max_iter=maxit, mf0=None, 
                                                                    ops=ops, hmft_tol=10**-8,
                                                                    lanczos_tol=10**-11,
                                                                    mf_cvg=False, every_other=True) 
    mf0 = mfs['lr'][i]
    get_mags(v, sl_mag_ops, i, 
             sl_mags['lr'], sl_2_mags['lr'], sl_4_mags['lr'])

In [ ]:
plt.figure(figsize=(4,3), dpi=400)
plt.scatter(J2s, energies['lr'][::-1], label='Left to right', marker='x')
plt.scatter(J2s, energies['rl'], label='Right to left', marker='+')
plt.scatter(J2s, energies['periodic'], label='Periodic')
plt.scatter(J2s, energies['inner'], label='Open')
plt.legend()

In [ ]:
plt.figure(figsize=(4,3), dpi=400)
m2 = sl_2_mags['lr']
m4 = sl_4_mags['lr']
norm = 1./(4./6)**2

m2z = (m2['A']['z'] + m2['B']['z'] + m2['C']['z'])/3
m4z = (m4['A']['z'] + m4['B']['z'] + m4['C']['z'])/3
Bz = .5*(3-m4z/(m2z**2))

m2x = (m2['A']['x'] + m2['B']['x'] + m2['C']['x'])/3
m4x = (m4['A']['x'] + m4['B']['x'] + m4['C']['x'])/3
Bx = .5*(3-m4x/(m2x**2))

m2y = (m2['A']['y'] + m2['B']['y'] + m2['C']['y'])/3
m4y = (m4['A']['y'] + m4['B']['y'] + m4['C']['y'])/3
By = .5*(3-m4y/(m2y**2))

m2xy = (m2['A']['x'] + m2['B']['x'] + m2['C']['x'])/3
m2xy += (m2['A']['y'] + m2['B']['y'] + m2['C']['y'])/3
m4xy = (m4['A']['x'] + m4['B']['x'] + m4['C']['x'])/3
m4xy += (m4['A']['y'] + m4['B']['y'] + m4['C']['y'])/3
Bxy = .5*(5-3*m4xy/(m2xy**2))


plt.plot(J2s[::-1], Bz, label=r'$B_z$')
plt.plot(J2s[::-1], Bx, label=r'$B_x$')
plt.plot(J2s[::-1], By, label=r'$B_y$')

plt.legend()

In [ ]:
corr_inds = [[0, 4, 8, 11],
             [2, 5, 8, 10],
             [3, 4, 5, 6]]
def get_corrs(v, op_str, inds, basis):
    i0 = inds[0]
    corrs = np.zeros(len(inds)-1, dtype=np.complex128)
    for i, ind in enumerate(inds[1:]):
        print(ind)
        op = quantum_operator({'static': [[op_str, [[.5, i0, ind]]]]}, basis=basis, check_symm=False, check_herm=False)
        op += quantum_operator({'static': [[op_str[::-1], [[.5, i0, ind]]]]}, basis=basis, check_symm=False, check_herm=False)

        corrs[i] = op.expt_value(v)
    return corrs

In [ ]:
interactions = {'local': {},
                'nearest': {'xx': 1, 'yy': 1, 'zz': 0},
                'n_nearest': {'xx': .15, 'yy': .15, 'zz': 0},
                'n_n_nearest': {}}
markers = ['+', 'x', '.']
e, v, mf, cvg = do_hmft(plaq, interactions, basis, 
                        max_iter=maxit, mf0=mf0, 
                        ops=ops, hmft_tol=10**-8,
                        lanczos_tol=10**-11,
                        mf_cvg=False, every_other=True)
# Hi = periodic_hamiltonian(plaq, interactions, basis, every_other=True)
# e, v = Hi.eigsh(k=1, which='SA', tol=10**-12)
# v = v[:,0]
plt.figure(figsize=(4,3), dpi=200)
print(e)
for i, inds in enumerate(corr_inds):
    print(inds)
    corrs = get_corrs(v, 'zz', inds, basis)
    plt.scatter(np.arange(3), np.real(corrs), marker=markers[i])
plt.ylabel(r'$C_{0r}^{zz}$')
plt.xlabel(r'$r$')

# Spin configs!

In [ ]:
ind_1_rl = np.argmin(np.abs(J2s-.15))
ind_2_rl = np.argmin(np.abs(J2s-.2))
steps=49

plt.figure(figsize=(6,5), dpi=400)
az = .5*np.sqrt(3)

plt.subplot(2,2,1)
draw_tri_lattice((-3, -4*az), 10, 10)
r0s = [[0,0], plaq['vs'][0], plaq['vs'][1], -plaq['vs'][0], -plaq['vs'][1],
       plaq['vs'][1] + plaq['vs'][0], -plaq['vs'][1] - plaq['vs'][0],
       plaq['vs'][1] - plaq['vs'][0], -plaq['vs'][1] + plaq['vs'][0]
      ]
for r0 in r0s:
    draw_cluster(plaq, r0, color='gray')
    plot_spins(plaq, mfs['rl'][ind_1_rl], r0)
plt.xlim(-2, 5)
plt.ylim(-2, 5)
plt.xticks([])
plt.yticks([])
plt.title(r'RL, $J_2 = {}$'.format(np.round(J2s[ind_1_rl], 3)))
print(energies['rl'][ind_1_rl])


plt.subplot(2,2,2)
draw_tri_lattice((-3, -4*az), 10, 10)
r0s = [[0,0], plaq['vs'][0], plaq['vs'][1], -plaq['vs'][0], -plaq['vs'][1],
       plaq['vs'][1] + plaq['vs'][0], -plaq['vs'][1] - plaq['vs'][0],
       plaq['vs'][1] - plaq['vs'][0], -plaq['vs'][1] + plaq['vs'][0]
      ]
for r0 in r0s:
    draw_cluster(plaq, r0, color='gray')
    plot_spins(plaq, mfs['rl'][ind_2_rl], r0)
plt.xlim(-2, 5)
plt.ylim(-2, 5)
plt.xticks([])
plt.yticks([])
plt.title(r'RL, $J_2 = {}$'.format(np.round(J2s[ind_2_rl], 3)))
print(energies['rl'][ind_2_rl])



plt.subplot(2,2,3)
draw_tri_lattice((-3, -4*az), 10, 10)
r0s = [[0,0], plaq['vs'][0], plaq['vs'][1], -plaq['vs'][0], -plaq['vs'][1],
       plaq['vs'][1] + plaq['vs'][0], -plaq['vs'][1] - plaq['vs'][0],
       plaq['vs'][1] - plaq['vs'][0], -plaq['vs'][1] + plaq['vs'][0]
      ]
for r0 in r0s:
    draw_cluster(plaq, r0, color='gray')
    plot_spins(plaq, mfs['lr'][steps-ind_1_rl], r0)
plt.xlim(-2, 5)
plt.ylim(-2, 5)
plt.xticks([])
plt.yticks([])
plt.title(r'LR $J_2 = {}$'.format(np.round(J2s[::-1][steps-ind_1_rl], 3)))
print(energies['lr'][steps-ind_1_rl])


plt.subplot(2,2,4)
draw_tri_lattice((-3, -4*az), 10, 10)
r0s = [[0,0], plaq['vs'][0], plaq['vs'][1], -plaq['vs'][0], -plaq['vs'][1],
       plaq['vs'][1] + plaq['vs'][0], -plaq['vs'][1] - plaq['vs'][0],
       plaq['vs'][1] - plaq['vs'][0], -plaq['vs'][1] + plaq['vs'][0]
      ]
for r0 in r0s:
    draw_cluster(plaq, r0, color='gray')
    plot_spins(plaq, mfs['lr'][steps-ind_2_rl], r0)
plt.xlim(-2, 5)
plt.ylim(-2, 5)
plt.xticks([])
plt.yticks([])
plt.title(r'LR $J_2 = {}$'.format(np.round(J2s[::-1][steps-ind_2_rl], 3)))
print(energies['lr'][steps-ind_2_rl])

plt.show()